In [27]:
import numpy as np
import pandas as pd
from scipy.stats import expon, reciprocal
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.preprocessing import OneHotEncoder as skohe
from sklearn.svm import SVR

In [28]:
housing = pd.read_csv('../data/raw/housing.csv')

In [29]:
housing.head()

longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
0    -122.23     37.88                41.0        880.0           129.0   
1    -122.22     37.86                21.0       7099.0          1106.0   
2    -122.24     37.85                52.0       1467.0           190.0   
3    -122.25     37.85                52.0       1274.0           235.0   
4    -122.25     37.85                52.0       1627.0           280.0   

   population  households  median_income  median_house_value ocean_proximity  
0       322.0       126.0         8.3252            452600.0        NEAR BAY  
1      2401.0      1138.0         8.3014            358500.0        NEAR BAY  
2       496.0       177.0         7.2574            352100.0        NEAR BAY  
3       558.0       219.0         5.6431            341300.0        NEAR BAY  
4       565.0       259.0         3.8462            342200.0        NEAR BAY

In [30]:
housing.shape

(20640, 10)

In [31]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [32]:
housing.isnull().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

In [33]:
housing.describe()

longitude      latitude  housing_median_age   total_rooms  \
count  20640.000000  20640.000000        20640.000000  20640.000000   
mean    -119.569704     35.631861           28.639486   2635.763081   
std        2.003532      2.135952           12.585558   2181.615252   
min     -124.350000     32.540000            1.000000      2.000000   
25%     -121.800000     33.930000           18.000000   1447.750000   
50%     -118.490000     34.260000           29.000000   2127.000000   
75%     -118.010000     37.710000           37.000000   3148.000000   
max     -114.310000     41.950000           52.000000  39320.000000   

       total_bedrooms    population    households  median_income  \
count    20433.000000  20640.000000  20640.000000   20640.000000   
mean       537.870553   1425.476744    499.539680       3.870671   
std        421.385070   1132.462122    382.329753       1.899822   
min          1.000000      3.000000      1.000000       0.499900   
25%        296.000000    787.000000    280.000000       2.563400   
50%        435.000000   1166.000000    409.000000       3.534800   
75%        647.000000   1725.000000    605.000000       4.743250   
max       6445.000000  35682.000000   6082.000000      15.000100   

       median_house_value  
count        20640.000000  
mean        206855.816909  
std         115395.615874  
min          14999.000000  
25%         119600.000000  
50%         179700.000000  
75%         264725.000000  
max         500001.000000

In [34]:
housing.describe(include='object')

ocean_proximity
count            20640
unique               5
top          <1H OCEAN
freq              9136

In [35]:
housing.ocean_proximity.value_counts()

ocean_proximity
<1H OCEAN     9136
INLAND        6551
NEAR OCEAN    2658
NEAR BAY      2290
ISLAND           5
Name: count, dtype: int64

In [36]:
# Split the data
X = housing.drop("median_house_value", axis=1)
y = housing["median_house_value"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

In [37]:
num_attribs = list(X.select_dtypes(include=[np.number]))
cat_attribs = ["ocean_proximity"]

In [38]:
# Create preprocessing pipeline
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', skohe(), cat_attribs)
])

In [39]:
# Create a full pipeline with preprocessing and SVR
svr_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('svr', SVR())
])

In [40]:
def get_feature_names_from_column_transformer(col_trans):
    """Get feature names from a sklearn column transformer.

    The `ColumnTransformer` class in `scikit-learn` supports taking in a
    `pd.DataFrame` object and specifying `Transformer` operations on columns.
    The output of the `ColumnTransformer` is a numpy array that can used and
    does not contain the column names from the original dataframe. The class
    provides a `get_feature_names` method for this purpose that returns the
    column names corr. to the output array. Unfortunately, not all
    `scikit-learn` classes provide this method (e.g. `Pipeline`) and still
    being actively worked upon.

	NOTE: This utility function is a temporary solution until the proper fix is
    available in the `scikit-learn` library.
    """

	# SimpleImputer has `add_indicator` attribute that distinguishes it from other transformers
    # Encoder had `get_feature_names` attribute that distinguishes it from other transformers
	# The last transformer is ColumnTransformer's 'remainder'


    col_name = []
    for transformer_in_columns in col_trans.transformers_:
        is_pipeline=0
        raw_col_name = list(transformer_in_columns[2])

        if isinstance(transformer_in_columns[1], Pipeline):
            # if pipeline, get the last transformer
            transformer = transformer_in_columns[1].steps[-1][1]
            is_pipeline=1
        else:
            transformer = transformer_in_columns[1]

        try:
            if isinstance(transformer, str):
                if transformer == "passthrough":
                    names = transformer._feature_names_in[raw_col_name].tolist()

                elif transformer == "drop":
                    names = []

                else:
                    raise RuntimeError(
                        f"Unexpected transformer action for unaccounted cols :"
                        f"{transformer} : {raw_col_name}"
                    )

            elif isinstance(transformer, skohe):
                names = list(transformer.get_feature_names(raw_col_name))

            elif isinstance(transformer, SimpleImputer) and transformer.add_indicator:
                missing_indicator_indices = transformer.indicator_.features_
                missing_indicators = [raw_col_name[idx] + '_missing_flag' for idx in missing_indicator_indices]

                names = raw_col_name + missing_indicators

            else:
                names = list(transformer.get_feature_names())

        except AttributeError as error:
              names = raw_col_name
        if is_pipeline:
            names=[f'{transformer_in_columns[0]}_{col_}'for col_ in names]
        col_name.extend(names)

    return col_name

In [42]:
X_train_prepared = preprocessor.fit_transform(X_train)

In [43]:
feature_names = get_feature_names_from_column_transformer(preprocessor)
print("Feature names after preprocessing: ", feature_names)

Feature names after preprocessing:  ['num_longitude', 'num_latitude', 'num_housing_median_age', 'num_total_rooms', 'num_total_bedrooms', 'num_population', 'num_households', 'num_median_income', 'ocean_proximity']


In [44]:
forest_reg = RandomForestRegressor(random_state=101)
forest_reg.fit(X_train_prepared, y_train)

feature_importances = forest_reg.feature_importances_

# #1

In [45]:
param_grid = [
    {'svr__kernel': ['linear'], 'svr__C': [0.1, 5, 50, 250]},
    {'svr__kernel': ['rbf'], 'svr__C': [0.5, 5, 50, 500], 'svr__gamma': [0.05, 0.1, 5, 50, 500]}
]

In [46]:
grid_search = GridSearchCV(svr_pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=2)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV] END .....................svr__C=0.1, svr__kernel=linear; total time=   5.7s
[CV] END .....................svr__C=0.1, svr__kernel=linear; total time=   5.7s
[CV] END .....................svr__C=0.1, svr__kernel=linear; total time=   6.1s
[CV] END .....................svr__C=0.1, svr__kernel=linear; total time=   5.5s
[CV] END .....................svr__C=0.1, svr__kernel=linear; total time=   5.7s
[CV] END .......................svr__C=5, svr__kernel=linear; total time=   5.8s
[CV] END .......................svr__C=5, svr__kernel=linear; total time=   5.6s
[CV] END .......................svr__C=5, svr__kernel=linear; total time=   5.6s
[CV] END .......................svr__C=5, svr__kernel=linear; total time=   5.5s
[CV] END .......................svr__C=5, svr__kernel=linear; total time=   5.5s
[CV] END ......................svr__C=50, svr__kernel=linear; total time=   5.5s
[CV] END ......................svr__C=50, svr__

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['longitude',
                                                                          'latitude',
                                                                          'housing_median_age',
                                                                          'total_rooms',
                                                                          'total_bedrooms',
                                                                          'population',
                                                                          'households',
                                                                          'median_income']),
                                                                        ('cat',
                                                                         OneHotEncoder(),
                                                                         ['ocean_proximity'])])),
                                       ('svr', SVR())]),
             param_grid=[{'svr__C': [0.1, 5, 50, 250],
                          'svr__kernel': ['linear']},
                         {'svr__C': [0.5, 5, 50, 500],
                          'svr__gamma': [0.05, 0.1, 5, 50, 500],
                          'svr__kernel': ['rbf']}],
             scoring='neg_mean_squared_error', verbose=2)

In [47]:
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print("Best parameters found: ", grid_search.best_params_)
print("Best RMSE: ", rmse)

Best parameters found:  {'svr__C': 250, 'svr__kernel': 'linear'}
Best RMSE:  73193.22888007555


Linear kernel seems better than the RBF kernel. Also the C value is the highest.

# #2

In [48]:
param_distributions = {
    'svr__kernel': ['linear', 'rbf'],
    'svr__C': reciprocal(20, 20000),
    'svr__gamma': expon(scale=1.0)
}

random_search = RandomizedSearchCV(svr_pipeline, param_distributions=param_distributions, n_iter=20, cv=5, scoring='neg_mean_squared_error', verbose=2, random_state=42)
random_search.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] END svr__C=265.8583788632432, svr__gamma=3.010121430917521, svr__kernel=linear; total time=   5.8s
[CV] END svr__C=265.8583788632432, svr__gamma=3.010121430917521, svr__kernel=linear; total time=   5.8s
[CV] END svr__C=265.8583788632432, svr__gamma=3.010121430917521, svr__kernel=linear; total time=   6.0s
[CV] END svr__C=265.8583788632432, svr__gamma=3.010121430917521, svr__kernel=linear; total time=   5.7s
[CV] END svr__C=265.8583788632432, svr__gamma=3.010121430917521, svr__kernel=linear; total time=   5.5s
[CV] END svr__C=4366.193678104918, svr__gamma=0.9084469696321253, svr__kernel=rbf; total time=   7.3s
[CV] END svr__C=4366.193678104918, svr__gamma=0.9084469696321253, svr__kernel=rbf; total time=   7.3s
[CV] END svr__C=4366.193678104918, svr__gamma=0.9084469696321253, svr__kernel=rbf; total time=   7.3s
[CV] END svr__C=4366.193678104918, svr__gamma=0.9084469696321253, svr__kernel=rbf; total time=   7.2s
[CV] END s

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('num',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(strategy='median')),
                                                                                               ('scaler',
                                                                                                StandardScaler())]),
                                                                               ['longitude',
                                                                                'latitude',
                                                                                'housing_median_age',
                                                                                'total_rooms',
                                                                                'total_bedrooms',
                                                                                'population',
                                                                                'households',
                                                                                'median_income']),
                                                                              ('cat',
                                                                               OneHotEncoder(),
                                                                               ['ocean_proximity'])])),
                                             ('svr', SVR())]),
                   n_iter=20,
                   param_distributions={'svr__C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f82a05a5a30>,
                                        'svr__gamma': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f82a0546d30>,
                                        'svr__kernel': ['linear', 'rbf']},
                   random_state=42, scoring='neg_mean_squared_error',
                   verbose=2)

In [49]:
best_model_random = random_search.best_estimator_
y_pred_random = best_model_random.predict(X_test)
mse_random = mean_squared_error(y_test, y_pred_random)
rmse_random = np.sqrt(mse_random)

print("Best parameters found: ", random_search.best_params_)
print("Best RMSE: ", rmse_random)

Best parameters found:  {'svr__C': 16683.860588281546, 'svr__gamma': 0.26497040005002437, 'svr__kernel': 'rbf'}
Best RMSE:  60300.17906874081


In [50]:
random_search.best_params_['svr__kernel']

'rbf'

Now we can see RBF kernal having high C value and samll gamma is better

# #3

In [51]:
k = 5

In [52]:
class TopKFeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, feature_importances, k):
        self.feature_importances = feature_importances
        self.k = k

    def fit(self, X, y=None):
        self.top_k_indices_ = np.sort(np.argpartition(self.feature_importances, -self.k)[-self.k:])
        return self

    def transform(self, X):
        return X[:, self.top_k_indices_]

In [53]:
preparation_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('feature_selection', TopKFeatureSelector(feature_importances, k))
])

housing_prepared_top_k_features = preparation_pipeline.fit_transform(X_train)

print(housing_prepared_top_k_features[:3])

[[ 0.72026503 -0.6763631   1.54231204  0.4507375   1.        ]
 [-1.1298396   1.91635176  1.14432091 -0.84694133  1.        ]
 [-0.78575006  1.47722527 -0.68643828  0.15674859  1.        ]]


# #4

In [54]:
kernel=random_search.best_params_['svr__kernel']
C=random_search.best_params_['svr__C']
gamma=random_search.best_params_['svr__gamma']

kernel, C, gamma

('rbf', 16683.860588281546, 0.26497040005002437)

In [55]:
prepare_and_final_predict_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('feature_selection', TopKFeatureSelector(feature_importances, k)),
    ('svr', SVR(kernel=random_search.best_params_['svr__kernel'],
                C=random_search.best_params_['svr__C'],
                gamma=random_search.best_params_['svr__gamma']))
])

In [56]:
prepare_and_final_predict_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['longitude', 'latitude',
                                                   'housing_median_age',
                                                   'total_rooms',
                                                   'total_bedrooms',
                                                   'population', 'households',
                                                   'median_income']),
                                                 ('cat', OneHotEncoder(),
                                                  ['ocean_proximity'])])),
                ('feature_selection',
                 TopKFeatureSelector(feature_importances=array([0.10952413, 0.10556588, 0.04966397, 0.02449959, 0.0212316 ,
       0.03097043, 0.01890579, 0.48318974, 0.00416368, 0.14383039,
       0.00049997, 0.00070186, 0.00725296]),
                                     k=5)),
                ('svr', SVR(C=16683.860588281546, gamma=0.26497040005002437))])

# #5

In [57]:
param_grid = [{
    'preprocessor__num__imputer__strategy': ['mean', 'median', 'most_frequent'],
    'feature_selection__k': list(range(1, len(feature_importances) + 1)),
    'preprocessor__num__scaler': [StandardScaler(), None]
}]

grid_search_prep = GridSearchCV(prepare_and_final_predict_pipeline, param_grid, cv=3,
                                scoring='neg_mean_squared_error', verbose=2)
grid_search_prep.fit(X_train, y_train)

Fitting 3 folds for each of 78 candidates, totalling 234 fits
[CV] END feature_selection__k=1, preprocessor__num__imputer__strategy=mean, preprocessor__num__scaler=StandardScaler(); total time=   5.5s
[CV] END feature_selection__k=1, preprocessor__num__imputer__strategy=mean, preprocessor__num__scaler=StandardScaler(); total time=   5.5s
[CV] END feature_selection__k=1, preprocessor__num__imputer__strategy=mean, preprocessor__num__scaler=StandardScaler(); total time=   5.5s
[CV] END feature_selection__k=1, preprocessor__num__imputer__strategy=mean, preprocessor__num__scaler=None; total time=   5.5s
[CV] END feature_selection__k=1, preprocessor__num__imputer__strategy=mean, preprocessor__num__scaler=None; total time=   5.5s
[CV] END feature_selection__k=1, preprocessor__num__imputer__strategy=mean, preprocessor__num__scaler=None; total time=   5.5s
[CV] END feature_selection__k=1, preprocessor__num__imputer__strategy=median, preprocessor__num__scaler=StandardScaler(); total time=   5.7s

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['longitude',
                                                                          'latitude',
                                                                          'housing_median_age',
                                                                          'total_rooms',
                                                                          'total_bedrooms',
                                                                          'population',
                                                                          'households',
                                                                          'median_income']),
                                                                        ('cat',
                                                                         OneHotEncoder(),
                                                                         ['ocean_proximity...
       0.03097043, 0.01890579, 0.48318974, 0.00416368, 0.14383039,
       0.00049997, 0.00070186, 0.00725296]),
                                                            k=5)),
                                       ('svr',
                                        SVR(C=16683.860588281546,
                                            gamma=0.26497040005002437))]),
             param_grid=[{'feature_selection__k': [1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                   10, 11, 12, 13],
                          'preprocessor__num__imputer__strategy': ['mean',
                                                                   'median',
                                                                   'most_frequent'],
                          'preprocessor__num__scaler': [StandardScaler(),
                                                        None]}],
             scoring='neg_mean_squared_error', verbose=2)

In [58]:
print("Best parameters with varied preprocessing options: ", grid_search_prep.best_params_)
print("Best RMSE with varied preprocessing options: ", np.sqrt(-grid_search_prep.best_score_))

Best parameters with varied preprocessing options:  {'feature_selection__k': 11, 'preprocessor__num__imputer__strategy': 'median', 'preprocessor__num__scaler': StandardScaler()}
Best RMSE with varied preprocessing options:  58504.86835078475


Seems like median is better imputing strategy and 11 features are useful.